In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
import json

In [6]:
filename = "run-500-200-20-part3_337.json"

path = os.path.join("results", filename)
with open(path, "r") as f:
    results = json.load(f)

In [19]:
results.keys()
experiments = results['experiments']

algorithms = {}
for algo in results['algorithms_info']:
    algorithms[algo['name']] = algo

print(results.keys())

print(experiments[0].keys())

print(experiments[0]['coverage_dist'])
print(experiments[0]['results']['Majority'])

dict_keys(['n_sources', 'n_dataitems', 'n_distinct', 'algorithms_info', 'experiments'])
dict_keys(['coverage_dist', 'truth_dist', 'distinct_dist', 'spread_dist', 'optimal', 'n_claims', 'iteration_index', 'results'])
{'name': 'TruncExponential', 'lmbda': 0.5, 'flipped': True}
{'time': 0.038550333999999964, 'scores': [1.0, 200]}


In [40]:
def results_to_dataframe(filename, folder="results"):
    path = os.path.join("results", filename)
    with open(path, "r") as f:
        results = json.load(f)
        
    dictionary = {
        'n_sources': results['n_sources'],
        'n_dataitems': results['n_dataitems'],
        'n_distinct': results['n_distinct'],
    }

    algorithms = {}
    for algo in results['algorithms_info']:
        algorithms[algo['name']] = algo

    for experiment in results['experiments']:
        for dist in ['coverage_dist', 'truth_dist', 'distinct_dist', 'spread_dist']:
            if 'flipped' in experiment[dist] and experiment[dist]['flipped']:
                dictionary[dist] = "F_" + experiment[dist]['name']
            else:
                dictionary[dist] = experiment[dist]['name']

        dictionary['optimal_perc_score'] = experiment['optimal'][0]
        dictionary['optimal_score'] = experiment['optimal'][1]
        dictionary['n_claims'] = experiment['n_claims']
        dictionary['iteration_index'] = experiment['iteration_index'] if 'iteration_index' in experiment else -1

        for key, value in experiment['results'].items():
            dictionary['algorithm'] = key
            dictionary['time'] = value['time']
            dictionary['score_perc'] = value['scores'][0]
            dictionary['score'] = value['scores'][1]
            print(dictionary)
        return

In [41]:
filename = "run-500-200-20-part3_337.json"
results_to_dataframe(filename)

{'n_sources': 500, 'n_dataitems': 200, 'n_distinct': 20, 'coverage_dist': 'F_TruncExponential', 'truth_dist': 'F_TruncExponential', 'distinct_dist': 'TruncExponential', 'spread_dist': 'Uniform', 'optimal_perc_score': 1.0, 'optimal_score': 200, 'n_claims': 54628, 'iteration_index': 337, 'algorithm': 'Majority', 'time': 0.038550333999999964, 'score_perc': 1.0, 'score': 200}
{'n_sources': 500, 'n_dataitems': 200, 'n_distinct': 20, 'coverage_dist': 'F_TruncExponential', 'truth_dist': 'F_TruncExponential', 'distinct_dist': 'TruncExponential', 'spread_dist': 'Uniform', 'optimal_perc_score': 1.0, 'optimal_score': 200, 'n_claims': 54628, 'iteration_index': 337, 'algorithm': 'TruthFinder', 'time': 0.53186775, 'score_perc': 1.0, 'score': 200}
{'n_sources': 500, 'n_dataitems': 200, 'n_distinct': 20, 'coverage_dist': 'F_TruncExponential', 'truth_dist': 'F_TruncExponential', 'distinct_dist': 'TruncExponential', 'spread_dist': 'Uniform', 'optimal_perc_score': 1.0, 'optimal_score': 200, 'n_claims': 5

In [ ]:
dictionary = {
    'n_sources': results['n_sources'],
    'n_dataitems': results['n_dataitems'],
    'n_distinct': results['n_distinct'],
    'coverage_dist':,
    'truth_dist':,
    'distinct_dist':,
    'spread_dist':,
    'optimal':,
    'n_claims':,
    'iteration_index':,
    'algorithm':,
    'time':,
    'score':,
    'score_perc':,
    
}